<a href="https://colab.research.google.com/github/FranklineMisango/Deep_Learning_Projects/blob/main/LimeLocale_Finance_NLP_Beta_2.0.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This project encompasses the LimeLocale's Finance API beta official AI that assists the team in Research in payment patterns in Fintech
#As a starting step, It currently automates the Team's research on Stock and Crypto in China's GBA

In [ ]:
#Baseline_Setup_and_dependencies
#Pegasus Tokenizer, NLP for summarization
#BS4 scraps the web for news
import sentencepiece
import tensorflow
import requests
import re
import tensorflow
from transformers import PegasusTokenizer,PegasusForConditionalGeneration, TFPegasusForConditionalGeneration
from bs4 import BeautifulSoup
from transformers import pipeline


In [ ]:
#Loading and saving the summarization models  from the Hugging face model
'''
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)
'''

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
model = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")

In [ ]:
#Test article summarization for tesla Undercovering in china 
#Used from yFinance#
#TODO-Fix the Issues on r responses and paragraph responses from yahoo finance texts
#TODO --> Mass testing on various URLS
'''
url = "https://ca.finance.yahoo.com/news/tesla-erases-50-billion-market-143450930.html"
r = requests.get(url)
soup = BeautifulSoup(r.text , 'html.parser')
paragraphs = soup.find_all('p')
text = [paragraph.text for paragraph in paragraphs]
words  = ''.join(text).split(' ')[:400]
ARTICLE = ' '.join(words)
# Truncate input sequence to the maximum length supported by the tokenizer
max_input_length = tokenizer.model_max_length
input_ids = tokenizer.encode(ARTICLE, return_tensors='pt', max_length=max_input_length, truncation=True)
output = model.generate(input_ids, num_beams = 5, max_length = 55, early_stopping = True)
summary = tokenizer.decode(output[0], skip_special_tokens=True)
'''

In [ ]:
#summary

'Services PMI expected to fall to 54.5 in February. Tesla’s investor day failed to live up to hype'

This Code slices focus on the Automation of more than one stock symbol

In [ ]:
#Building a News and Sentiment Pipleline
#Test with company stock names --Edited to user preference
monitored_tickers = ['GME' , 'TSLA', 'BTC'] # will be edited to allow user input

#Search for stock news using Google and Yahoo Finance
#TODO--> Figure out how to allow a universal Web scrapping instead of relying solely on Yahoo finance. ie investopedia, Bloomberg LP etc
def search_for_stock_news_urls(ticker):
  search_url = 'https://www.google.com/search?q=yahoo+finance+{}&client=firefox-b-e&sxsrf=AJOqlzVLATO8V6-Wc70eLzYFgTNtHvp7Vw:1678435886874&source=lnms&tbm=nws&sa=X&ved=2ahUKEwiZzsL29ND9AhX4tlYBHXb2APQQ_AUoAXoECAEQAw&biw=1366&bih=617&dpr=1'.format(ticker)
  r = requests.get(search_url)
  soup =  BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all('a')
  hrefs = [link['href'] for link in atags]
  return hrefs

raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'GME': ['/?sa=X&ved=0ahUKEwiJlKeMotj9AhWiVTABHX-PAKcQOwgC',
  '/search?q=yahoo+finance+GME&tbm=nws&client=firefox-b-e&sa=X&biw=1366&bih=617&ie=UTF-8&gbv=1&sei=EsEOZInDE6KrwbkP_56CuAo',
  '/search?q=yahoo+finance+GME&client=firefox-b-e&biw=1366&bih=617&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiJlKeMotj9AhWiVTABHX-PAKcQ_AUIBSgA',
  '/search?q=yahoo+finance+GME&client=firefox-b-e&biw=1366&bih=617&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwiJlKeMotj9AhWiVTABHX-PAKcQ_AUIBygC',
  '/search?q=yahoo+finance+GME&client=firefox-b-e&biw=1366&bih=617&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiJlKeMotj9AhWiVTABHX-PAKcQ_AUICCgD',
  '/search?q=yahoo+finance+GME&client=firefox-b-e&biw=1366&bih=617&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiJlKeMotj9AhWiVTABHX-PAKcQ_AUICSgE',
  'https://maps.google.com/maps?q=yahoo+finance+GME&client=firefox-b-e&sxsrf=AJOqlzVLATO8V6-Wc70eLzYFgTNtHvp7Vw:1678435886874&biw=1366&bih=617&dpr=1&um=1&ie=UTF-8&sa=X&ved=0ahUKEwiJlKeMotj9AhWiVTABHX-PAKcQ_AUICigF',
  '/searc

In [ ]:
#Strip out the unwanted URLs--> policies,accounts, preferences etc
excluded_list = ['maps', 'policies', 'preferences', 'support', 'accounts']
def strip_unwanted_urls(urls, excluded_list):
    final_val = []
    for url in urls:
        if 'https://' in url and not any(excluded_word in url for excluded_word in excluded_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            final_val.append(res)
    return list(set(final_val))

cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], excluded_list) for ticker in monitored_tickers}
cleaned_urls


{'GME': ['https://finance.yahoo.com/news/dada-nexus-limited-nasdaq-dada-120253785.html',
  'https://finance.yahoo.com/news/wall-street-watchdog-set-adopt-054326135.html',
  'https://finance.yahoo.com/news/strength-seen-gamestop-gme-14-144602277.html',
  'https://finance.yahoo.com/news/amc-entertainment-holdings-amc-stock-100647547.html',
  'https://ca.finance.yahoo.com/video/pitney-bowes-most-compelling-investment-161143490.html',
  'https://finance.yahoo.com/news/gamestop-reports-third-quarter-fiscal-210500302.html',
  'https://finance.yahoo.com/news/my-meme-stock-fiasco-153417561.html',
  'https://finance.yahoo.com/news/why-game-stop-stock-is-probably-dead-money-for-a-while-according-to-one-analyst-180851839.html',
  'https://ca.finance.yahoo.com/news/wallstreetbets-stocks-where-now-104500429.html',
  'https://finance.yahoo.com/news/zacks-investment-ideas-feature-highlights-135001128.html',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BGME%26client%3Dfirefox-b-e%26sxsrf%3DAJ

In [ ]:
#Seach and scrape cleaned URLs
def scrape_and_process(urls):
    ARTICLES = []
    for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ''.join(text).split(' ')[:400]
        ARTICLE = ''.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'GME': ["DadaNexusLimited(NASDAQ:DADA)Q42022EarningsCallTranscriptMarch9,2023Operator:Goodmorning,ladiesandgentlemen,andthankyouforstandingbyforDada'sFourthQuarter2022EarningsConferenceCall..Iwillnowturnthemeetingovertoyourhostfortoday'scall,Ms.CarolineDong,HeadofInvestorRelationsforDada.Pleaseproceed,Caroline.CarolineDong:Thankyou,operator.Hello,everyone,andthankyouforjoiningourfourthquarter2022earningsconferencecall.OnthecalltodayfromDada,wehaveMr.JeffHuijianHe,PresidentandMr.BeckChen,CFO;andMr.JunYang,Co-FounderandCTO.Mr.Hewilltalkaboutouroperationsandcompanyhighlights.ThenMr.Chenwilldiscussthefinancialsandguidance.PleasecarefullynotethatduringtheQ&Asession,JeffwillanswerquestionsinChineseandtheconsecutivetranslationwillbeprovided.Incaseofanydiscrepancybetweentheoriginalremarksandthetranslatedversionstatementsintheoriginalremarksshouldprevail.Beforewebegin,I'dliketoremindyouthatthisconferencecallcontainsforward-lookingstatements.Pleaserefertoourlatestsafeharborstatementintheearning

In [ ]:
#Refer to the solo article solo testing section to understand the model
def summarize_all_articles(articles):
    summaries = []
    for article in articles:
      max_input_length = tokenizer.model_max_length
      input_ids = tokenizer.encode(article, return_tensors='pt', max_length=max_input_length, truncation=True)
      output = model.generate(input_ids, num_beams = 5, max_length = 55, early_stopping = True)
      summary_1 = tokenizer.decode(output[0], skip_special_tokens=True)
      summaries.append(summary_1)
    return summaries
 
summaries = {ticker:summarize_all_articles(articles[ticker]) for ticker in monitored_tickers}
summaries


{'GME': ['Company to release unaudited results for the fourth quarter of 2022 after the US markets close.',
  'Regulators adopt new rules for stock trades, crypto assets.',
  'Engineers are working to resolve the issue.',
  'Company’s debt-to-equity ratio falls to 34.7%, lowest since 2009.',
  'KurtWolf, Hestia Capital, joins Yahoo Finance live to discuss mail business.',
  'Engineers are working to resolve the issue.',
  'Engineers are working to resolve the issue.',
  'Engineers are working to resolve the issue.',
  'Two of the most popular stocks that internet investors were buying were GameStop andAMC.',
  'GameStopCorp, Tesla, Hershey, Yum China, Box all in spotlight.',
  'Your browsing history will not be shared with third parties.'],
 'TSLA': ['Auto-Tires-Trucks sector falls as much as 9.62%, draggingTesla down.',
  'Musk bought the shares in open market transactions on Thursday.',
  'Musk says he’s open to the idea of Twitter buying Silicon Bank. ‘Whatanopportunity,’ says real-

In [ ]:
#Adding Sentiment Analysis to give a Negative/Positive score
#-VE means the market is not good for the product
#+VE means the market is good for the product/investment
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

scores ={ticker:sentiment(summaries[ticker])for ticker in monitored_tickers}
scores

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'GME': [{'label': 'NEGATIVE', 'score': 0.9874897599220276},
  {'label': 'NEGATIVE', 'score': 0.8910480737686157},
  {'label': 'POSITIVE', 'score': 0.9737206101417542},
  {'label': 'NEGATIVE', 'score': 0.9993143081665039},
  {'label': 'POSITIVE', 'score': 0.7437362670898438},
  {'label': 'POSITIVE', 'score': 0.9737206101417542},
  {'label': 'POSITIVE', 'score': 0.9737206101417542},
  {'label': 'POSITIVE', 'score': 0.9737206101417542},
  {'label': 'NEGATIVE', 'score': 0.9462580680847168},
  {'label': 'POSITIVE', 'score': 0.9995995163917542},
  {'label': 'NEGATIVE', 'score': 0.999546468257904}],
 'TSLA': [{'label': 'NEGATIVE', 'score': 0.9997748732566833},
  {'label': 'POSITIVE', 'score': 0.9684739708900452},
  {'label': 'POSITIVE', 'score': 0.8099250793457031},
  {'label': 'POSITIVE', 'score': 0.9997739195823669},
  {'label': 'POSITIVE', 'score': 0.9939701557159424},
  {'label': 'POSITIVE', 'score': 0.8909408450126648},
  {'label': 'NEGATIVE', 'score': 0.9893509745597839},
  {'label': '

In [ ]:
#Printing the individual scores for each 

print(summaries['TSLA'][0] , scores['BTC'][0]['label'], scores['BTC'][0]['score'])

Auto-Tires-Trucks sector falls as much as 9.62%, draggingTesla down. NEGATIVE 0.9990084767341614


The final slice exports our results to a csv file for further analyses 

In [45]:
#Step-1 is to summarize the blueprint labels for the CSV
def create_output_array(summaries, scores, urls):
  output = []
  for ticker in monitored_tickers:
    for counter in range(len(summaries[ticker])):
      output_this = [
      ticker,
      summaries[ticker][counter],
      scores[ticker][counter],
      urls[ticker][counter],
    ]
    output.append(output_this)
  return output

final_output = create_output_array(summaries,scores,cleaned_urls)
final_output

[['GME',
  'Your browsing history will not be shared with third parties.',
  {'label': 'NEGATIVE', 'score': 0.999546468257904},
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BGME%26client%3Dfirefox-b-e%26sxsrf%3DAJOqlzVLATO8V6-Wc70eLzYFgTNtHvp7Vw:1678435886874%26source%3Dlnms%26tbm%3Dnws%26sa%3DX%26ved%3D2ahUKEwiZzsL29ND9AhX4tlYBHXb2APQQ_AUoAXoECAEQAw%26biw%3D1366%26bih%3D617%26dpr%3D1%26pccc%3D1'],
 ['TSLA',
  'MetaPlatforms, Tesla, NFLX,GOOGL,NVDA, 888-349-8884 are in spotlight.',
  {'label': 'POSITIVE', 'score': 0.9971967935562134},
  'https://finance.yahoo.com/news/fangman-big-cap-growth-stocks-205608932.html'],
 ['BTC',
  'Total marketcapitalization for cryptos fell4.6% to$1.02trillion. Silvergate says it will delay filing of annual financial statements',
  {'label': 'NEGATIVE', 'score': 0.999563992023468},
  'https://finance.yahoo.com/news/bitcoin-fell-nearly-5-thursday-night-its-largest-one-hour-drop-since-ftx-140957690.html']]

In [51]:
final_output

[['Ticker', 'Summary', 'Label', 'Confidence', 'Url'],
 ['Ticker', 'Summary', 'Label', 'Confidence', 'Url'],
 ['Ticker', 'Summary', 'Label', 'Confidence', 'Url'],
 ['Ticker', 'Summary', 'Label', 'Confidence', 'Url'],
 ['Ticker', 'Summary', 'Label', 'Confidence', 'Url'],
 ['GME',
  'Your browsing history will not be shared with third parties.',
  {'label': 'NEGATIVE', 'score': 0.999546468257904},
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BGME%26client%3Dfirefox-b-e%26sxsrf%3DAJOqlzVLATO8V6-Wc70eLzYFgTNtHvp7Vw:1678435886874%26source%3Dlnms%26tbm%3Dnws%26sa%3DX%26ved%3D2ahUKEwiZzsL29ND9AhX4tlYBHXb2APQQ_AUoAXoECAEQAw%26biw%3D1366%26bih%3D617%26dpr%3D1%26pccc%3D1'],
 ['TSLA',
  'MetaPlatforms, Tesla, NFLX,GOOGL,NVDA, 888-349-8884 are in spotlight.',
  {'label': 'POSITIVE', 'score': 0.9971967935562134},
  'https://finance.yahoo.com/news/fangman-big-cap-growth-stocks-205608932.html'],
 ['BTC',
  'Total marketcapitalization for cryptos fell4.6% to$1.02trillion. Silvergate says it wi

In [50]:
#Exporting the results to 
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Confidence', 'Url'])
import csv
with open('assetsummaries.csv', mode='w', newline='') as f:
  csv_writer = csv.writer(f, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
  csv_writer.writerows(final_output)